In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path = list(set(sys.path))

from common.evaluate import make_vis_env, test_env

from flow.multiagent_envs import MultiWaveAttenuationMergePOEnv
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from ray.tune import run_experiments

import gym, ray
from ray.rllib.agents.ppo import PPOAgent, DEFAULT_CONFIG
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

benchmark_name = 'multi_merge'

In [2]:
alg_run = "PPO"
num_rollouts = 50
num_cpus = 63
gae_lambda = 0.97
step_size = 5e-4

In [3]:
ray.init(num_cpus=num_cpus, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-18_21-50-35_727/logs.
Waiting for redis server at 127.0.0.1:49950 to respond...
Waiting for redis server at 127.0.0.1:45114 to respond...
Starting the Plasma object store with 13.355121049 GB memory using /dev/shm.


{'node_ip_address': '169.237.32.118',
 'object_store_addresses': ['/tmp/ray/session_2019-04-18_21-50-35_727/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-18_21-50-35_727/sockets/raylet'],
 'redis_address': '169.237.32.118:49950',
 'webui_url': ''}

In [4]:
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.flow_params
horizon = flow_params['env'].horizon

def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

create_env, env_name = make_create_env(params=flow_params, version=0)

In [5]:
config = DEFAULT_CONFIG.copy()

In [6]:
config["num_workers"] = min(num_cpus, num_rollouts) - 1
config["train_batch_size"] = horizon * num_rollouts
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [100, 50, 25]
config["observation_filter"] = "NoFilter"

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json
config['env_config']['run'] = alg_run

In [7]:
# Register as rllib env
register_env(env_name, create_env)

In [8]:
exp_tag = {
    "run": alg_run,
    "env": env_name,
    "config": {
        **config
    },
    "checkpoint_freq": 25,
    "max_failures": 999,
    "stop": {
        "training_iteration": 500
    },
    "num_samples": 1,
}

In [ ]:
trials = run_experiments({
        flow_params["exp_tag"]: exp_tag
    })

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/63 CPUs, 0/1 GPUs
Memory usage on this node: 6.9/33.4 GB

Created LogSyncer for /headless/ray_results/multi_merge/PPO_MultiWaveAttenuationMergePOEnv-v0_0_2019-04-18_21-50-37nluq_gep -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/1 GPUs
Memory usage on this node: 6.9/33.4 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING

